This is mainly to just test RAG solutions for this type of problem. I will maybe try to implement a custom graphrag later.

In [83]:
from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils import is_flash_attn_2_available

import pandas as pd
from openai import OpenAI

In [84]:
device = "cuda" if torch.cuda.is_available() else "cpu"

Creating vector database embeddings

In [85]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

# Load in sentences
df = pd.read_csv('mathinfo.csv')

# Sentences are encoded/embedded by calling model.encode()
embeddings = embedding_model.encode(df['Description'].tolist(), convert_to_tensor=True, device=device)
df['embedding'] = embeddings.tolist()

In [86]:
embeddings = torch.tensor(df['embedding'].tolist(), dtype=torch.float32, device=device)
embeddings.shape

torch.Size([10, 384])

In [87]:
query = "What is the derivative of x^2?"
print(f"Query: {query}")

query_embedding = embedding_model.encode(query, convert_to_tensor=True, device=device)

dot_scores = torch.nn.functional.cosine_similarity(query_embedding, embeddings)
top_k = 5
top_k_indices = torch.topk(dot_scores, top_k).indices
dot_scores_list = dot_scores.tolist()
print(f"Top {top_k} most similar sentences:")
for i in top_k_indices:
    print(f"Score: {dot_scores_list[i]:.4f}, Operation: {df['Operation'][i.item()]}")

Query: What is the derivative of x^2?
Top 5 most similar sentences:
Score: 0.6161, Operation: Derivative
Score: 0.3148, Operation: Square Root
Score: 0.2861, Operation: Integration
Score: 0.1576, Operation: Exponentiation
Score: 0.1270, Operation: Absolute Value


In [88]:
def cosine_similarity(query_embedding, embeddings):
    dot_scores = torch.nn.functional.cosine_similarity(query_embedding, embeddings)
    return dot_scores, dot_scores.tolist()

def retrieve_relevant_resources(query, df, embedding_model, top_k=5):
    query_embedding = embedding_model.encode(query, convert_to_tensor=True, device=device)
    names = df['Operation'].tolist()
    descriptions = df['Description'].tolist()
    text = [f"{name}: {description}" for name, description in zip(names, descriptions)]
    embeddings = embedding_model.encode(text, convert_to_tensor=True, device=device)
    dot_scores, dot_list = cosine_similarity(query_embedding, embeddings)
    top_k_indices = torch.topk(dot_scores, top_k).indices
    return top_k_indices, dot_list

In [89]:
if (is_flash_attn_2_available()) and (torch.cuda.get_device_capability(0)[0] >= 8):
    attn_implementation = "flash_attention_2"
else:
    attn_implementation = "sdpa"
print(f"Using {attn_implementation} attention")

Using sdpa attention


In [90]:
model_id = "gemma-2-9b-it"
print(f"Using model: {model_id}")

Using model: gemma-2-9b-it


In [97]:
client = OpenAI(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio"
)

In [98]:
system_prompt = "You are a math tutor helping a student with calculus homework"
prompt = "What is the derivative of x^2?"

messages = [{"role": "system", "content": system_prompt}, 
            {"role": "user", "content": prompt}]
response = client.chat.completions.create(
    model=model_id,
    messages=messages
)
results = response.choices[0].message.content
print(results)

Great question!  

The derivative of x² is **2x**. 

Here's why:

* **The Power Rule:** We use a rule called the power rule for derivatives. The power rule states that the derivative of x<sup>n</sup> is nx<sup>n-1</sup>.

* **Applying the Rule:** In this case, n = 2. So, applying the power rule:
   *  d/dx (x²) = 2x<sup>(2-1)</sup> = 2x¹ = 2x


Let me know if you'd like to see more examples or have any other calculus questions!


In [103]:
prompt = """Water is being poured into a cylindrical tank at a constant rate. The volume of water in the tank after t minutes is given by the function:

V(t) = 4t^2 + 10

Question:
At what rate is the volume of water changing after 3 minutes?"""
system_preprompt = """You are a machine learning model that summarizes math problems for students. 
The following is a math problem. Return only the operation NAMES necessary to solve the problem in order from most relevant to least relevant, but DO NOT solve the problem: """
messages = [{"role": "system", "content": system_preprompt},
            {"role": "user", "content": prompt}]
response = client.chat.completions.create(
    model=model_id,
    messages=messages
)
preprompt = response.choices[0].message.content
print(preprompt)

system_prompt_1 = "You are a math tutor helping a student with math homework. The following are related operations to the question in order from most relevant to least relevant: "
system_prompt_2 = "Please break the question down step-by-step for the student. Here is the student's question: "
# system_prompt_1 = "You are a math tutor helping a student with math homework. The following are the related operations to the question and their sympy representations in order from most relevant to least relevant: "
# system_prompt_2 = "Please break the question down step-by-step for the student and give the sympy representation of the question at each step. Here is the student's question: "

top_k = 5
top_k_indices, dot_list = retrieve_relevant_resources(preprompt, df, embedding_model, top_k=top_k)
print(f"Top {top_k} most relevant operations:")
for i in top_k_indices:
    print(f"Score: {dot_list[i]:.4f}, Operation: {df['Operation'][i.item()]}")

operations_prompt = ""
for i in top_k_indices:
    operations_prompt += f"{df['Operation'][i.item()]}\n"
    # operations_prompt += f"{df['Operation'][i.item()]} | {df['Sympy Representation'][i.item()]}\n"

appended_system_prompt = f"{system_prompt_1}\n{operations_prompt}\n{system_prompt_2}"
print(appended_system_prompt)

messages = [{"role": "system", "content": appended_system_prompt},
            {"role": "user", "content": prompt}]
response = client.chat.completions.create(
    model=model_id,
    messages=messages
)
results = response.choices[0].message.content
print(results)

Here are the operation names in order of relevance:

1. **Differentiation** 
2. **Substitution**  

Top 5 most relevant operations:
Score: 0.4660, Operation: Derivative
Score: 0.3025, Operation: Integration
Score: 0.2377, Operation: Exponentiation
Score: 0.1922, Operation: Multiplication
Score: 0.1529, Operation: Division
You are a math tutor helping a student with math homework. The following are related operations to the question in order from most relevant to least relevant: 
Derivative
Integration
Exponentiation
Multiplication
Division

Please break the question down step-by-step for the student. Here is the student's question: 
Here's how to break down this problem step-by-step:

**1. Understand the Problem:**

* We have a function, V(t), that tells us the volume of water in the tank at any given time (t).  
* The question asks for the *rate* of change of the volume – essentially how fast the volume is increasing or decreasing at a specific moment (3 minutes in this case).

**2. R